# Compute mean of observation for a specific year

In [1]:
# modules importation
from trends_functions import *
from warnings import filterwarnings
filterwarnings('ignore')

In [2]:
# run observations
varss = ['od550aer', 'od550lt1aer', 'od550gt1aer', 'ang4487aer', 'concpm2p5', 'concpm10', 'concso4', 'scatc550dryaer', 'absc550aer']
regions = ['EUROPE', 'NAMERICA', 'SAMERICA', 'NAFRICA', 'SAFRICA', 'ASIA', 'AUSTRALIA']

In [3]:
OBS_MAP = {}
for var in varss:
    if var in ['concpm10', 'concpm25']:
        print('read EBAS')
        EBAS = read_EBAS()
    else:
        EBAS = None
    # computation parameters
    params = get_params()
    params['kind'] = 'obs'
    params['min_dobs'] = 300
    fill_params(params, var)
    if var in ['scatc550dryaer', 'absc550aer']:
        params['period'] = '2000-2018'
    else:
        params['period'] = '2000-2014'
    params['var'] = var
    
    obs_source = params['source']
    reader = pya.io.ReadUngridded(obs_source)
    obs_data = reader.read(vars_to_retrieve=var)
    
    print('compute trend')
    _, OBS_MAP[var], _ = process_trend(
        obs_data, params,
        plot=False, show_plot=False, save_plot=False, write_json=False, EBAS=EBAS
    )

compute trend
no flags
no flags EUROPE station:  ZvenigoroddsEi_GCWD
no flags ASIA station:  Zhongshan_UnivityDh
no flags NAMERICA station:  Yumaowknife_AuroraC NAMERICA station:  Calipso_Sterling_PO NAMERICA station:  DRAGON_EaglePoint NAMERICA station:  IMPROVE-MammothCave
no flags SAMERICA station:  Veracruz_MAX_MEXFOPSAMERICA station:  Concepcion
no flags NAFRICA station:  Zinder_DMNporttutea
region:  SAFRICA station:  Walvis_Bay_airport
Less than 7 points in selected period
no flags
no flags AUSTRALIA station:  Tinga_Tinganand_QLD
compute trendD station:  ZvenigorodportityaP WORLD station:  Calipso_Mardela_Spr WORLD station:  Lecce_University
no flags
no flags EUROPE station:  ZvenigorodMOW1erCmD
no flags ASIA station:  Zhongshan_UnivityDh ASIA station:  DRAGON_Kyungil_Univ ASIA station:  Ieodo_Station ASIA station:  Taihu
no flags NAMERICA station:  Yumaowknife_AuroraC NAMERICA station:  Calipso_W_Mardela
no flags SAMERICA station:  UdeConcepcion-CEFOP
no flags NAFRICA station:  

## put this in a dataframe

In [4]:
#'concpm25', 'concpm10', 'concso4', 'scatc550dryaer', 'absc550aer'

In [5]:
regions.remove('SAFRICA')

In [8]:
thead = ['']+regions
tbody = []

for var in varss:
    period = '2000-2014'
    if var in ['concpm2p5', 'concpm10', 'scatc550dryaer', 'absc550aer']:
        fmt = '{:2.1f}'
        if var in ['scatc550dryaer', 'absc550aer']:
            period = '2000-2018'
    else:
        fmt = '{:3.2f}'
    print(var)
    fill_params(params, var)
    unit = params['unit']
    if unit == '':
        str_unit = ''
    else:
        str_unit = ' ('+unit+')'
    line = [params['ylabel']+str_unit] + [fmt.format(OBS_MAP[var][region]['trends']['trends'][period]['data'][0]) if region in OBS_MAP[var] and 'trends' in OBS_MAP[var][region]['trends'] and period in OBS_MAP[var][region]['trends']['trends'] else '-' for region in regions]
    tbody.append(line)

    df = pd.DataFrame(tbody, columns=thead)

od550aer
od550lt1aer
od550gt1aer
ang4487aer
concpm2p5
concpm10
concso4
scatc550dryaer
absc550aer


In [9]:
with pd.option_context("max_colwidth", 1000):
    latable = df.to_latex(index=False, longtable=False, escape=False)
    #replace rule with hline for ACP
    latable = latable.replace('toprule','tophline')
    latable = latable.replace('midrule','middlehline')
    latable = latable.replace('bottomrule','bottomhline')
    print(latable)

\begin{tabular}{lllllll}
\tophline
                    & EUROPE & NAMERICA & SAMERICA & NAFRICA &  ASIA & AUSTRALIA \\
\middlehline
                AOD &   0.17 &     0.10 &     0.15 &    0.26 &  0.37 &      0.10 \\
            AOD<1µm &   0.15 &     0.08 &     0.11 &    0.11 &  0.22 &      0.04 \\
            AOD>1µm &   0.03 &     0.02 &     0.03 &    0.10 &  0.09 &      0.03 \\
                 AE &   1.43 &     1.48 &     1.26 &    0.70 &  1.16 &      1.00 \\
     PM2.5 (µg.m-3) &   12.3 &      6.9 &        - &     9.0 &     - &         - \\
      PM10 (µg.m-3) &   16.8 &     12.4 &        - &    19.7 &     - &         - \\
       SO4 (µg.m-3) &   2.01 &     1.45 &        - &    2.98 &  1.97 &         - \\
 Scat. Coef. (1/Mm) &   34.4 &     23.4 &        - &       - &     - &         - \\
  Abs. Coef. (1/Mm) &    6.3 &      2.6 &        - &       - &     - &         - \\
\bottomhline
\end{tabular}

